In [121]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Pedestrian sensor data file
pedestrian_sensor_locations_path = "../Datasets/pedestrian-counting-system-sensor-locations.csv"

# Read the pedestrian sensor data file
pedestrian_sensor_locations = pd.read_csv(pedestrian_sensor_locations_path)

# Import the OpenWeatherMap API key
#from api_keys import city_of_melbourne_api_key

pedestrian_sensor_locations_df = pd.DataFrame(pedestrian_sensor_locations)
pedestrian_sensor_locations_df

,location_id,Sensor_description,sensor_name,Installation_date,Note,location_type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
0,1,Bourke Street Mall (North),Bou292_T,2009-03-24,NaN,Outdoor,A,East,West,-37.813494,144.965153,"-37.81349441, 144.96515323"
1,3,Melbourne Central,Swa295_T,2009-03-25,NaN,Outdoor,A,North,South,-37.811015,144.964295,"-37.81101524, 144.96429485"
2,7,Birrarung Marr,FedPed_T,2014-12-17,NaN,Outdoor,A,East,West,-37.818629,144.971694,"-37.81862929, 144.97169395"
3,9,Southern Cross Station,Col700_T,2009-03-23,NaN,Outdoor,A,East,West,-37.819830,144.951026,"-37.81982992, 144.95102555"
4,12,New Quay,NewQ_T,2009-01-21,NaN,Outdoor,A,East,West,-37.814580,144.942924,"-37.81457988, 144.94292398"
...,...,...,...,...,...,...,...,...,...,...,...,...
103,111,Harbour Esplanade,HarbBlix_T,2021-03-25,NaN,Outdoor Blix,A,NaN,NaN,-37.814426,144.944366,"-37.81442577, 144.94436558"
104,115,Shrine of rememberance-North-East,ShrinNE_T,2022-10-14,NaN,Outdoor Blix,A,NaN,NaN,-37.829998,144.973768,"-37.82999797, 144.97376842"
105,119,Pop Up Library - Crossbar - Federation Square,PopLibFed_T,2022-01-25,NaN,Indoor,A,In,Out,-37.817882,144.969446,"-37.81788176, 144.96944609"
106,121,Royal Park BBQ near the playground and public ...,RoyBlix_T,2023-01-18,Royal Park BBQ near the playground and public ...,Outdoor Blix,A,NaN,NaN,-37.795052,144.951046,"-37.79505209, 144.95104583"


In [122]:
import hvplot.pandas
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts


sensors_map = pedestrian_sensor_locations_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600,
    color = "Status",
    alpha = 0.5,
    hover_cols=["location_id", "Sensor_description"]
)
print(f"Number of sensors in the list: {len(pedestrian_sensor_locations_df)}")
sensors_map

Number of sensors in the list: 108


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Status,location_id,Sensor_description)

In [123]:
# Filter rows with "Outdoor" value in "location_type" column
outdoor_pedestrian_sensors_df = pedestrian_sensor_locations_df[pedestrian_sensor_locations_df["location_type"] == "Outdoor"]

# List of sensors in Melbourne CBD
location_ids = [1, 2, 3, 4, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 30, 36, 39, 40, 41, 45, 47, 48, 49, 51, 52, 53, 55, 56, 58, 59, 61, 62, 63, 65, 66, 67, 68, 69, 71, 72, 73, 75, 79, 84, 88, 89, 98, 99, 107, 108, 109, 113] 

cbd_pedestrian_sensors_df = outdoor_pedestrian_sensors_df.loc[outdoor_pedestrian_sensors_df["location_id"].isin(location_ids)]

cbd_pedestrian_sensors_df = cbd_pedestrian_sensors_df.reset_index()
print(f"Number of sensors in the list: {len(cbd_pedestrian_sensors_df)}")
cbd_pedestrian_sensors_df


Number of sensors in the list: 46


,index,location_id,Sensor_description,sensor_name,Installation_date,Note,location_type,Status,Direction_1,Direction_2,Latitude,Longitude,Location
0,0,1,Bourke Street Mall (North),Bou292_T,2009-03-24,NaN,Outdoor,A,East,West,-37.813494,144.965153,"-37.81349441, 144.96515323"
1,1,3,Melbourne Central,Swa295_T,2009-03-25,NaN,Outdoor,A,North,South,-37.811015,144.964295,"-37.81101524, 144.96429485"
2,5,17,Collins Place (South),Col15_T,2009-03-30,Device is upgraded in 26/02/2020,Outdoor,A,East,West,-37.813625,144.973236,"-37.81362543, 144.97323591"
3,6,18,Collins Place (North),Col12_T,2009-03-30,NaN,Outdoor,A,East,West,-37.813449,144.973054,"-37.81344862, 144.97305353"
4,7,20,Chinatown-Lt Bourke St (South),LtB170_T,2013-09-06,NaN,Outdoor,A,East,West,-37.811729,144.968247,"-37.81172914, 144.9682466"
5,8,21,Bourke St-Russell St (West),Rus180_T,2013-09-02,NaN,Outdoor,A,North,South,-37.812447,144.967788,"-37.81244703, 144.96778757"
6,9,23,Spencer St-Collins St (South),Col623_T,2013-09-02,NaN,Outdoor,A,East,West,-37.819093,144.954527,"-37.81909256, 144.95452748"
7,11,36,Queen St (West),Que85_T,2015-01-20,NaN,Outdoor,A,North,South,-37.816525,144.961211,"-37.81652527, 144.96121062"
8,13,40,Lonsdale St-Spring St (West),Spr201_T,2015-01-19,NaN,Outdoor,A,North,South,-37.809993,144.972276,"-37.80999341, 144.97227587"
9,15,55,Elizabeth St-La Trobe St (East),Eli380_T,2018-07-19,NaN,Outdoor,A,North,South,-37.809889,144.961343,"-37.80988941, 144.96134331"


In [124]:
cbd_sensors_map = cbd_pedestrian_sensors_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "CartoLight",
    frame_width = 800,
    frame_height = 600,
    color = "Status",
    alpha = 0.5,
    hover_cols=["location_id", "Sensor_description"]
)

print(f"Number of sensors in the list: {len(cbd_pedestrian_sensors_df)}")
cbd_sensors_map

Number of sensors in the list: 46


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Status,location_id,Sensor_description)

In [125]:
# Define the list of location IDs from pedestrian_sensor_locations_df
location_ids = cbd_pedestrian_sensors_df['location_id'].tolist()

# Set the API base URL
url = 'https://data.melbourne.vic.gov.au/api/'

# Define an empty list to fetch the pedestrian count for each sensor
pedestrian_count = []

# Create counters
record_count = 1
set_count = 1

# Define the endpoint
endpoint = '/records/1.0/search/'

# Loop through all the location_ids in our list to pedestrian count
for location_id in location_ids:
    # Define the API parameters
    params = {
        'dataset': 'pedestrian-counting-system-past-hour-counts-per-minute',
        'q': '',
        'sort': 'sensingdatetime',
        'facet': 'locationid',
        'facet': 'sensingdatetime',
        'refine.locationid': location_id
    }
    # Send the API request
    response = requests.get(url + endpoint, params=params)

    # Check the status code of the response
    if response.status_code == 200:
        # Extract the pedestrian count from the response
        pedestrian_count.append(response.json()['nhits'])
        # Print the record count and location ID
        print(f"Record {record_count} of Set {set_count} | Location ID: {location_id} | Pedestrian Count: {response.json()['nhits']}")
    else:
        # Print an error message if the API request fails
        print(f"Error: API request failed for Location ID {location_id}")

    # Increment the record count
    record_count += 1

Record 1 of Set 1 | Location ID: 1 | Pedestrian Count: 37010
Record 2 of Set 1 | Location ID: 3 | Pedestrian Count: 43972
Record 3 of Set 1 | Location ID: 17 | Pedestrian Count: 32552
Record 4 of Set 1 | Location ID: 18 | Pedestrian Count: 26383
Record 5 of Set 1 | Location ID: 20 | Pedestrian Count: 34447
Record 6 of Set 1 | Location ID: 21 | Pedestrian Count: 0
Record 7 of Set 1 | Location ID: 23 | Pedestrian Count: 35277
Record 8 of Set 1 | Location ID: 36 | Pedestrian Count: 33193
Record 9 of Set 1 | Location ID: 40 | Pedestrian Count: 31529
Record 10 of Set 1 | Location ID: 55 | Pedestrian Count: 43870
Record 11 of Set 1 | Location ID: 61 | Pedestrian Count: 37636
Record 12 of Set 1 | Location ID: 62 | Pedestrian Count: 37041
Record 13 of Set 1 | Location ID: 63 | Pedestrian Count: 30209
Record 14 of Set 1 | Location ID: 69 | Pedestrian Count: 25893
Record 15 of Set 1 | Location ID: 71 | Pedestrian Count: 10222
Record 16 of Set 1 | Location ID: 84 | Pedestrian Count: 8070
Record 1

In [133]:
import requests
import pandas as pd

# Define the list of location IDs from pedestrian_sensor_locations_df
location_ids = cbd_pedestrian_sensors_df['location_id'].tolist()

# Set the API base URL
url = 'https://data.melbourne.vic.gov.au/api/'

# Define an empty list to fetch the pedestrian count for each sensor
pedestrian_count = []

# Create counters
record_count = 1
set_count = 1

# Define the endpoint
endpoint = '/records/1.0/search/'

# Loop through all the location_ids in our list to pedestrian count
for location_id in location_ids:
    # Define the API parameters
    params = {
        'dataset': 'pedestrian-counting-system-past-hour-counts-per-minute',
        'q': '',
        'sort': 'sensingdatetime',
        'facet': 'locationid',
        'facet': 'sensingdatetime',
        'format': 'json',
        'timezone': 'UTC',
        'refine.locationid': location_id,
        #'SensingDateTime': '2022-04-11T18:11:00+00:00'
    }

    # Send the API request
    response = requests.get(url + endpoint, params=params)

    # Check the status code of the response
    if response.status_code == 200:
        # Extract the pedestrian count from the response
        records = response.json().get('records', [])
        count = sum(int(record['fields']['total_of_directions']) for record in records)
        pedestrian_count.append(count)
        # Print the record count and location ID
        print(f"Record {record_count} of Set {set_count} | Location ID: {location_id} | Pedestrian Count: {count}")
    else:
        # Print an error message if the API request fails
        print(f"Error: API request failed for Location ID {location_id}")
        print(f"Status code: {response.status_code}")
        print(f"Response text: {response.text}")

    # Increment the record count
    record_count += 1


Record 1 of Set 1 | Location ID: 1 | Pedestrian Count: 34
Record 2 of Set 1 | Location ID: 3 | Pedestrian Count: 129
Record 3 of Set 1 | Location ID: 17 | Pedestrian Count: 26
Record 4 of Set 1 | Location ID: 18 | Pedestrian Count: 20
Record 5 of Set 1 | Location ID: 20 | Pedestrian Count: 29
Record 6 of Set 1 | Location ID: 21 | Pedestrian Count: 0
Record 7 of Set 1 | Location ID: 23 | Pedestrian Count: 15
Record 8 of Set 1 | Location ID: 36 | Pedestrian Count: 18
Record 9 of Set 1 | Location ID: 40 | Pedestrian Count: 14
Record 10 of Set 1 | Location ID: 55 | Pedestrian Count: 114
Record 11 of Set 1 | Location ID: 61 | Pedestrian Count: 30
Record 12 of Set 1 | Location ID: 62 | Pedestrian Count: 27
Record 13 of Set 1 | Location ID: 63 | Pedestrian Count: 54
Record 14 of Set 1 | Location ID: 69 | Pedestrian Count: 14
Record 15 of Set 1 | Location ID: 71 | Pedestrian Count: 15
Record 16 of Set 1 | Location ID: 84 | Pedestrian Count: 741
Record 17 of Set 1 | Location ID: 108 | Pedestria